In [16]:
import numpy as np
import gensim
import string
import re
import math


from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
from keras.utils.data_utils import get_file


from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#PreProcessing

In [13]:
!ls
%cd drive/
!ls
%cd "My Drive"
# !ls
%cd assigmment_nlp
!ls


max_length = 40
with open("speeches.txt", 'r', encoding='utf-8') as myfile:
  c = myfile.read()
  c=c.lower()
  token = sent_tokenize(c)
  token = [re.sub(r'[^a-z0-9]'," ", i) for i in token]
  
sent_token = ["<s> " + i + "</s>" for i in token]
# token

sentences = [sent.split()[:max_length] for sent in sent_token]

train = sentences[:int(len(sentences)*0.8)]
test = sentences[int(len(sentences)*0.8):]

print(test)

sentences = train


word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)


austin.txt  speeches.txt
[Errno 2] No such file or directory: 'drive/'
/content/drive/My Drive/assigmment_nlp
austin.txt  speeches.txt
[Errno 2] No such file or directory: 'My Drive'
/content/drive/My Drive/assigmment_nlp
[Errno 2] No such file or directory: 'assigmment_nlp'
/content/drive/My Drive/assigmment_nlp
austin.txt  speeches.txt
[['<s>', 'so', 'no', 'that', 's', 'not', 'true', '</s>'], ['<s>', 'well', 'putin', 'was', 'interviewed', 'and', 'i', 'was', 'interviewed', 'last', 'night', '</s>'], ['<s>', 'i', 'thought', 'his', 'interview', 'i', 'though', 'charlie', 'rose', 'did', 'an', 'excellent', 'job', '</s>'], ['<s>', 'but', 'i', 'thought', 'his', 'was', 'a', 'softer', 'interview', '</s>'], ['<s>', 'scott', 'pelley', 'interviewed', 'me', '</s>'], ['<s>', 'i', 'thought', 'scott', 'was', 'terrific', '</s>'], ['<s>', 'i', 'mean', 'it', 'was', 'a', 'tough', 'interview', 'but', 'i', 'thought', 'he', 'was', 'very', 'fair', 'and', 'i', 'thought', 'the', 'piece', 'was', 'very', 'good', 

#Getting word vectors from word2vec

In [4]:
trained_weights = word_model.wv.vectors
trained_vocabs = list(word_model.wv.vocab)
# word_model["</s>"]
vocab_size = word_model.wv.vectors.shape[0]
embedding_size = word_model.wv.vectors.shape[1]
# print(embedding_size)

print(vocab_size)

def word2idx(word):
  return word_model.wv.vocab[word].index       #https://github.com/carpedm20/word2vec-tensorflow/blob/master/model.py 
def idx2word(idx):                             #reference 
  return word_model.wv.index2word[idx]



# print(trained_weights[0])

# print(word2idx('thank'))

# word_model.wv.vocab['</s>'].index

word_model.wv.index2word[122]

5177


'things'

#Model Initialization LSTM

In [5]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[trained_weights]))
lstm_model.add(LSTM(units=embedding_size))
lstm_model.add(Dense(units=vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#Getting Input and Output Samples for Model

In [6]:
print(len(sentences))
train_x = np.zeros([len(sentences), max_length], dtype=np.int32)
# print(train_x)
train_y = np.zeros([len(sentences)], dtype=np.int32)
print(train_y.shape)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
# print('train_x shape:', train_x.shape)
# print('train_y shape:', train_y.shape)

13120
(13120,)


#Training the Model LSTM

In [7]:
lstm_model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
13120/13120 [==============================] - 9s 660us/step - loss: 1.9957
Epoch 2/20
13120/13120 [==============================] - 7s 554us/step - loss: 0.1345
Epoch 3/20
13120/13120 [==============================] - 7s 559us/step - loss: 0.1115
Epoch 4/20
13120/13120 [==============================] - 7s 557us/step - loss: 0.0965
Epoch 5/20
13120/13120 [==============================] - 7s 552us/step - loss: 0.0860
Epoch 6/20
13120/13120 [==============================] - 7s 551us/step - loss: 0.0778
Epoch 7/20
13120/13120 [==============================] - 7s 548us/step - loss: 0.0709
Epoch 8/20
13120/13120 [==============================] - 7s 559us/step - loss: 0.0641
Epoch 9/20
13120/13120 [==============================] - 7s 544us/step - loss: 0.0573
Epoch 10/20
13120/13120 [==============================] - 7s 548us/step - loss: 0.0507
Epoch 11/20
13120/13120 [==========

#Generating Sentences

In [8]:
start_arr = ['They are','so','here','we','with']
for i in start_arr:

  
  len_of_seq = 3
  word_idxs = [word2idx(word) for word in i.lower().split()]



  for i in range(len_of_seq):
    preds = lstm_model.predict(x=np.array([word_idxs]))

    probas = np.random.multinomial(1, preds[0], 1)

    idx = np.argmax(probas)

    word_idxs.append(idx)

  print(' '.join(idx2word(idx) for idx in word_idxs))


they are burden regular based
so bowe ordered mouth
here medicaid yourself carter
we reagan political committed
with food lives received


#Model for RNN

In [0]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[trained_weights]))
rnn_model.add(SimpleRNN(units=embedding_size))
rnn_model.add(Dense(units=vocab_size))
rnn_model.add(Activation('softmax'))
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#Training RNN Model

In [10]:
rnn_model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)

Epoch 1/20
13120/13120 [==============================] - 2s 188us/step - loss: 2.0329
Epoch 2/20
13120/13120 [==============================] - 2s 159us/step - loss: 0.1578
Epoch 3/20
13120/13120 [==============================] - 2s 156us/step - loss: 0.1218
Epoch 4/20
13120/13120 [==============================] - 2s 159us/step - loss: 0.1039
Epoch 5/20
13120/13120 [==============================] - 2s 158us/step - loss: 0.0923
Epoch 6/20
13120/13120 [==============================] - 2s 158us/step - loss: 0.0836
Epoch 7/20
13120/13120 [==============================] - 2s 157us/step - loss: 0.0767
Epoch 8/20
13120/13120 [==============================] - 2s 160us/step - loss: 0.0702
Epoch 9/20
13120/13120 [==============================] - 2s 160us/step - loss: 0.0642
Epoch 10/20
13120/13120 [==============================] - 2s 160us/step - loss: 0.0582
Epoch 11/20
13120/13120 [==============================] - 2s 158us/step - loss: 0.0523
Epoch 12/20
13120/13120 [================

#Generating Sentences from RNN Model

In [11]:
start_arr = ['They are','so','here','we','with']
for i in start_arr:

  
  len_of_seq = 3
  word_idxs = [word2idx(word) for word in i.lower().split()]



  for i in range(len_of_seq):
    preds = rnn_model.predict(x=np.array([word_idxs]))

    probas = np.random.multinomial(1, preds[0], 1)

    idx = np.argmax(probas)

    word_idxs.append(idx)

  print(' '.join(idx2word(idx) for idx in word_idxs))

they are family wanting squad
so groups page comprehension
here excavation hundreds beneficiary
we witnessed born citizenship
with out flag gen


#Perplexity of LSTM Model


In [21]:

sent_data=test

word_model = gensim.models.Word2Vec(sent_data, size=100, min_count=1, iter=100)
trained_vocabs = word_model.wv.vocab
trained_weights = np.array(word_model.wv.vectors)
vocab_size = word_model.wv.vectors.shape[0]
embedding_size = word_model.wv.vectors.shape[1]

      
N=len(sent_data)
perplexity=0
for i, line in enumerate(sent_data):
  x=line[:-1]
  y=line[-1]
  windex_x=[word_model.wv.vocab[word].index for word in x]
  preds=lstm_model.predict(x=np.array([windex_x]))
  idx=word_model.wv.vocab[y].index
  prob=preds[0][idx]
  perplexity=perplexity-(math.log(prob)/N)
perplexity=math.exp(perplexity)

perplexity

28.64481190694615